### Solution: The *Tower of Hanoi* problem

The basic recursive formulation for $n$ disks is to transfer $(n-1)$ of those at the top to the spare pole, transfer the $n^\text{th}$ disk to the destination, and then transfer the $n-1$ stack to the destination.

In [2]:
def operation(n, src, dest):
  aux = 3 - src - dest
  if n == 1:
    print(f"Move disk 1 from rod {src} to rod {dest}.")
    return
  operation(n - 1, src, aux)
  print(f"Move disk {n} from rod {src} to rod {dest}.")
  operation(n - 1, aux, dest)

print("Operations for n = 2:")
operation(2, 0, 2)

print("Operations for n = 3:")
operation(3, 0, 2)

print("Operations for n = 4:")
operation(4, 0, 2)

Operations for n = 2:
Move disk 1 from rod 0 to rod 1.
Move disk 2 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.
Operations for n = 3:
Move disk 1 from rod 0 to rod 2.
Move disk 2 from rod 0 to rod 1.
Move disk 1 from rod 2 to rod 1.
Move disk 3 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 0.
Move disk 2 from rod 1 to rod 2.
Move disk 1 from rod 0 to rod 2.
Operations for n = 4:
Move disk 1 from rod 0 to rod 1.
Move disk 2 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.
Move disk 3 from rod 0 to rod 1.
Move disk 1 from rod 2 to rod 0.
Move disk 2 from rod 2 to rod 1.
Move disk 1 from rod 0 to rod 1.
Move disk 4 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.
Move disk 2 from rod 1 to rod 0.
Move disk 1 from rod 2 to rod 0.
Move disk 3 from rod 1 to rod 2.
Move disk 1 from rod 0 to rod 1.
Move disk 2 from rod 0 to rod 2.
Move disk 1 from rod 1 to rod 2.


### Solution: Emulating partially-specialized functions

```
template<typename T> struct print_value_s{
  static void op(T const& x){
    std::cout << "Generic value of x = " << x << std::endl;
  }
};
template<typename T> struct print_value_s<TestClass<T>>{
  static void op(TestClass<T> const& x){
    std::cout << "TestClass<T>.x = " << x << std::endl;
  }
};
template<typename T> struct print_value_s<TestClass<TestClass<T>>>{
  static void op(TestClass<TestClass<T>> const& x){
    std::cout << "TestClass<TestClass<T>>.x = " << x << std::endl;
  }
};
template<typename T> void print_value(T const& x){ print_value_s<T>::op(x); }
// Without the bonus: Remove the static qualifiers, and define last function as
// template<typename T> void print_value(T const& x){ print_value_s<T>().op(x); }
```

### Solution: Finding out whether a public member function (value()) exists

```
template<typename T> struct has_call_value{
  struct invalid_call_type{};
  template <typename U> static constexpr auto test(int) -> decltype(&U::value);
  template <typename U> static constexpr auto test(...) -> invalid_call_type;
  static constexpr bool value = !std_ttraits::is_same_v<invalid_call_type, decltype(test<T>(0))>;
};
template<typename T> inline constexpr bool has_call_value_v = has_call_value<T>::value;
```

### Solution: CRTP performance

```
#include <chrono>
#include <iostream>


class Base{
public:
  virtual void increment(int j) = 0;
  virtual int value() = 0;
};
class Derived : public Base{
private:
  int v;
public:
  Derived() : v(0) {}
  void increment(int j){ v += j; }
  int value(){ return v; }
};


template<typename T> class Base_CRTP{
public:
  void increment(int j){ ((T*) this)->increment_impl(j); }
  int value(){ return ((T*) this)->value_impl(); }
};
class Derived_CRTP : public Base_CRTP<Derived_CRTP>{
private:
  int v;
public:
  Derived_CRTP() : v(0) {}
  void increment_impl(int j){ v += j; }
  int value_impl(){ return v; }
};


template<typename T> void test(unsigned long long int n, T* obj, const char* name){
  using namespace std;
  auto t1 = chrono::high_resolution_clock::now();
  for (unsigned long long int i=0; i<n; ++i) obj->increment(i);
  cout << name << " value: " << obj->value() << endl;
  auto t2 = chrono::high_resolution_clock::now();
  auto diff = chrono::duration_cast<chrono::microseconds>(t2-t1).count();
  cout << "- Duration: " << diff << " mus" << endl;
}

int main(){
  constexpr unsigned long long int n = 100000000;
  Base* d = new Derived();
  test(n, d, "Non-CRTP object");
  delete d;

  Base_CRTP<Derived_CRTP>* dc = new Derived_CRTP();
  test(n, dc, "CRTP object");
  delete dc;
}
```

With optimization flags `-O3` for `gcc`, you should observe that the CRTP implementation is a factor of about two faster.